# Como el trabajo con DASH no funciono se usara plotly para los graficos interactivos

In [ ]:
import plotly.graph_objs as go
fig= go.FigureWidget(
    data=[
        dict(
            type='scattergl',
            x=[1,2,3,4,5],
            y=[3,6,8,4,7],
            mode='markers',
        )
    ],
)
fig.layout.title="Ejemplo"
fig.layout.xaxis.title="X"
fig.layout.yaxis.title="Y"
fig2=fig
test=fig.add_scatter(x=[1,2,3,4,5],y=[9,8,9,8,9])
from ipywidgets import HBox,VBox
VBox([fig,fig2,HBox([fig2,fig])])

In [ ]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.io as pio

import os
import numpy as np
init_notebook_mode(connected=True)
N = 100
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
sz = np.random.rand(N)*30

fig = go.Figure()
fig.add_scatter(x=x,
                y=y,
                mode='markers',
                marker={'size': sz,
                        'color': colors,
                        'opacity': 0.6,
                        'colorscale': 'Viridis'
                       });
iplot(fig)

In [ ]:
import plotly.offline as off

off.init_notebook_mode(connected=True)

subject = ['Moe','Larry','Curly','Moe','Larry','Curly','Moe','Larry','Curly','Moe','Larry','Curly']
score = [1,6,2,8,2,9,4,5,1,5,2,8]

data = [dict(
  type = 'scatter',
  x = subject,
  y = score,
  mode = 'markers',
  transforms = [dict(
    type = 'groupby',
    groups = subject,
    styles = [
        dict(target = 'Moe', value = dict(marker = dict(color = 'blue'))),
        dict(target = 'Larry', value = dict(marker = dict(color = 'red'))),
        dict(target = 'Curly', value = dict(marker = dict(color = 'black')))
    ]
  )]
)]

off.iplot({'data': data}, validate=False)

# Para leer los datos directos del sistema se utilizaran las librerias para manejar archivos XML

In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree
import datetime
import matplotlib.pyplot as plt
import distutils
from distutils import util
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
import os
from ipywidgets import interactive, HBox, VBox


init_notebook_mode(connected=True)

def process_data(x):
    e = xml.etree.ElementTree.parse(x).getroot()
    df=pd.DataFrame(list(map(lambda y:list(map(lambda x: x.text,y)),e.getchildren())))
    cols=list(map(lambda x:x.tag,e.getchildren()[1].getchildren()))
    df.columns=cols
    df.index=df.iloc[:,0]
    #df=df.drop('usuarioapertura',axis=1)
    df=np.transpose(np.transpose(df).loc[~pd.isnull(np.transpose(df.iloc[1,:]))]).iloc[1:len(df),:]
    return df

In [2]:
df=process_data('cheques_I.xml')
df1=process_data('cheques_P.xml')
df2=process_data('cheques_T.xml')
df=df.append(df1).append(df2)
df=df.drop('usuarioapertura',axis=1)
df
cheqdet=process_data('cheqdet.xml')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




# DATOS DE PEDIDOS desde SQL

In [3]:
import pyodbc 
cnxn = pyodbc.connect(DRIVER='{SQL Server}',
                      Server='{VENTAS-6\\NATIONALSOFT}',
                      Database='{Softrestaurant95Centralizador}',
                      Trusted_Connection='yes',)


cursor = cnxn.cursor()

### Conversion datos extraidos de SQL en datos manejables

In [18]:
pedidosdet=pd.read_sql('select * from dbo.pedidosdetalle',cnxn)
sucursales={'01':1,'06':1,'16':1,'17':1,'10':2,'11':2,'12':2,'13':3,'14':3,'15':3,'18':4,'19':4,'20':4}
pedidosdet.idalmacen=pd.DataFrame({'idalmacen':pedidosdet.idalmacen.apply(lambda x:sucursales[x])})


pedidos=pd.read_sql('select * from dbo.pedidos',cnxn)
pedidos.idempresa=pd.DataFrame({'idempresa':pedidos.idempresa.apply(lambda x:int(x))})


### Convertir datos extraidos del archivo cheques en datos manejables

In [5]:
df['fecha']=list(map(lambda x:np.datetime64(x),df['fecha']))
df['cierre']=list(map(lambda x:np.datetime64(x),df['cierre']))
#x.astype('M8[ms]').astype('O').hour
df['total']=list(map(lambda x:float(x),df['total']))
df['totalalimentos']=list(map(lambda x:float(x),df['totalalimentos']))
df['totalbebidas']=list(map(lambda x:float(x),df['totalbebidas']))
df['totalotros']=list(map(lambda x:float(x),df['totalotros']))
df['totaldescuentoalimentos']=list(map(lambda x:float(x),df['totaldescuentoalimentos']))
df['totaldescuentobebidas']=list(map(lambda x:float(x),df['totaldescuentobebidas']))
df['totaldescuentobebidas']=list(map(lambda x:float(x),df['totaldescuentobebidas']))
df['efectivo']=list(map(lambda x:float(x),df['efectivo']))
df['tarjeta']=list(map(lambda x:float(x),df['tarjeta']))
df['nopersonas']=list(map(lambda x:int(x),df['nopersonas']))
df['cancelado']=list(map(lambda x:distutils.util.strtobool(x),df['cancelado']))
df['idempresa']=list(map(lambda x:int(x),df['idempresa']))

#df.columns.tolist()

### Convertir datos extraidos del archivo cheqdet en datos manejables

In [6]:
cheqdet['hora']=list(map(lambda x:np.datetime64(x),cheqdet['hora']))
#x.astype('M8[ms]').astype('O').hour
cheqdet['precio']=list(map(lambda x:float(x),cheqdet['precio']))
cheqdet['cantidad']=list(map(lambda x:float(x),cheqdet['cantidad']))
cheqdet['movimiento']=list(map(lambda x:float(x),cheqdet['movimiento']))


# DashBoard

In [160]:
def product_total(x):
    return np.sum(x.cantidad*x.precio)

class plotting:
    
    def __init__(self,df,cheqdet):
        self.fig=go.Figure()
        self.df=df
        self.original=df
        self.cheqdet=cheqdet
        self.Ocheqdet=cheqdet
    
        
    def update_Date(self,HOUR,MONTH):
        self.df=self.original
        self.df = self.df[[a and b for a,b in zip(list(map(lambda x: x.month==MONTH,self.df.fecha)),list(map(lambda x: x.hour==HOUR,self.df.fecha)))]]
        self.cheques_graph()
        
    def update_Suc(self,MONTH,YEAR):
        self.cheqdet=self.Ocheqdet
        self.cheqdet = self.cheqdet[[a and b for a,b in zip(list(map(lambda x: x.month==MONTH,self.cheqdet.hora)),list(map(lambda x: x.year==YEAR,self.cheqdet.hora)))]]
        self.cheqdet_graph()
        
    def cheques_graph(self):
        self.fig=go.Figure()
        self.fig.add_pie=self.fig.add_pie(labels=list(map(lambda x:x.weekday(),self.df.fecha)),values=self.df.total,);
        iplot(self.fig)
        
        self.Enterprise_pie=go.Figure()
        self.Enterprise_pie.add_pie=self.Enterprise_pie.add_pie(labels=self.df.idempresa,values=self.df.total,);
        iplot(self.Enterprise_pie)
    
    def cheqdet_graph(self):
        self.Product_pie=go.Figure()
        self.Product_pie.add_pie=self.Product_pie.add_pie(labels=self.cheqdet.idproducto,values=self.cheqdet.cantidad,);
        iplot(self.Product_pie)
        
        self.Product_top5=go.Figure()
        result=pd.DataFrame(self.cheqdet.groupby('idproducto').apply(product_total)).sort_values([0],ascending=[0]).head()
        self.Product_top5.add_table=self.Product_top5.add_table(header = dict(
            values = list(['top5','cantidad']),
            line = dict(color = 'black'),
            fill = dict(color='#C2D4FF'),
            align = 'center',
            font = dict(color = 'black', size = 12)
            ),
        cells = dict(
            values = [result.index,result[0]],
            align = 'center',
            font = dict(color = 'black', size = 11),
            fill = dict(color='#F5F8FF')
            ))
        iplot(self.Product_top5)
        
    def pedidosdet_graph(self):
        self.Pedidos_trace=go.Trace()
        
        



IndentationError: expected an indented block (<ipython-input-160-d9f9cc2483d8>, line 58)

In [ ]:
pie_chart=plotting(df,cheqdet)
freq_slider = interactive(pie_chart.update_Date, HOUR=(9, 22, 1),MONTH=(1, 12, 1))
suc_slider = interactive(pie_chart.update_Suc,MONTH=(1, 12, 1), YEAR=(2018, 2019, 1))
freq_slider
display(VBox([freq_slider,suc_slider]))

In [97]:
x=[False,True] 
y=[False,True]
z=[False,False]
[a and b for a,b in zip([a and b for a, b in zip(x, y)],z)]

[False, False]

# Traces

In [25]:
def series_values(temp,fecha):
    y=temp.groupby(fecha.dt.day)['total'].sum()
    x=pd.date_range(start=pd.Timestamp(2019,2,1), freq='D',periods=len(y))
    trace=go.Trace(x=x,y=y)
    return trace

temp=pedidos[[a and b for a,b in zip([a and b for a,b in zip(list(map(lambda x: x.month==2,pedidos.fechahoraenviado)),list(map(lambda x: x.year==2019,pedidos.fechahoraenviado)))],list(map(lambda x: x==2,pedidos.idempresa)))]]
trace=series_values(temp,temp.fechahoraenviado)
temp2=df[[a and b for a,b in zip([a and b for a,b in zip(list(map(lambda x: x.month==2,df.fecha)),list(map(lambda x: x.year==2019,df.fecha)))],list(map(lambda x: x==2,df.idempresa)))]]
trace2=series_values(temp2,temp2.fecha)


C:\ProgramData\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:672: DeprecationWarning:

plotly.graph_objs.Trace is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [26]:


data = [trace,trace2]

layout = dict(title = 'Venta vs Pedido',
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False)
             )

fig = dict(data=data, layout=layout)
iplot(fig, filename='styled-scatter')

In [ ]:
# codigo para acceder a la base de datos desde sql
# select * from [softrestaurant95pro].dbo.[cheques]